In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
a_cyrillic = ord('а')
alphabet_list = ''.join([chr(i).upper() for i in range(a_cyrillic,a_cyrillic+6)] + [chr(a_cyrillic+33)] + [chr(i) for i in range(a_cyrillic+6,a_cyrillic+32)])
alphabet_list = alphabet_list.upper()

In [15]:
# AIM: To Decrypt a text using MCMC approach. i.e. find decryption key which we will call cipher from now on.
import string
import math
import random

# This function takes as input a decryption key and creates a dict for key where each letter in the decryption key
# maps to a alphabet For example if the decryption key is "ПНСЕО...." this function will create a dict like {П:А,Н:Б,С:В....}
def create_cipher_dict(cipher):
    cipher_dict = {}
    for i in range(len(cipher)):
        cipher_dict[alphabet_list[i]] = cipher[i]
    return cipher_dict

# This function takes a text and applies the cipher/key on the text and returns text.
def apply_cipher_on_text(text,cipher):
    cipher_dict = create_cipher_dict(cipher) 
    text = list(text)
    newtext = ""
    for elem in text:
        if elem.upper() in cipher_dict:
            newtext+=cipher_dict[elem.upper()]
        else:
            newtext+=" "
    return newtext

# This function takes as input a path to a long text and creates scoring_params dict which contains the 
# number of time each pair of alphabet appears together
# Ex. {'AБ':234,'КЛ':2343,'ЖЗ':23 ..}
def create_scoring_params_dict(longtext_path):
    scoring_params = {}
    with open(longtext_path) as fp:
        for line in fp:
            data = list(line.strip())
            for i in range(len(data)-1):
                alpha_i = data[i].upper()
                alpha_j = data[i+1].upper()
                if alpha_i not in alphabet_list and alpha_i != " ":
                    alpha_i = " "
                if alpha_j not in alphabet_list and alpha_j != " ":
                    alpha_j = " "
                key = alpha_i+alpha_j
                if key in scoring_params:
                    scoring_params[key]+=1
                else:
                    scoring_params[key]=1
    return scoring_params

# This function takes as input a path to a long text and creates scoring_params normalized dict which contains the 
# normalized number of time each pair of alphabet appears together
def create_normalized_scoring_params_dict(longtext_path):
    scoring_params = {}
    with open(longtext_path) as fp:
        # text_len = 
        for line in fp:
            data = list(line.strip())
            for i in range(len(data)-1):
                alpha_i = data[i].upper()
                alpha_j = data[i+1].upper()
                if alpha_i not in alphabet_list and alpha_i != " ":
                    alpha_i = " "
                if alpha_j not in alphabet_list and alpha_j != " ":
                    alpha_j = " "
                key = alpha_i+alpha_j
                if key in scoring_params:
                    scoring_params[key]+=1
                else:
                    scoring_params[key]=1
    # normalization
    appearance_num = scoring_params.values()
    total_appearance_num = sum(appearance_num)
    scoring_params = {k: v / total_appearance_num for k, v in scoring_params.items()}
    return scoring_params

# This function takes as input a text and creates scoring_params dict which contains the 
# number of time each pair of alphabet appears together
# Ex. {'АБ':234,'КЛ':2343,'ЖЗ':23 ..}
def score_params_on_cipher(text):
    scoring_params = {}
    data = list(text.strip())
    for i in range(len(data)-1):
        alpha_i =data[i].upper()
        alpha_j = data[i+1].upper()
        if alpha_i not in alphabet_list and alpha_i != " ":
            alpha_i = " "
        if alpha_j not in alphabet_list and alpha_j != " ":
            alpha_j = " "
        key = alpha_i+alpha_j
        if key in scoring_params:
            scoring_params[key]+=1
        else:
            scoring_params[key]=1
    return scoring_params

# This function takes as input a text and creates scoring_params dict which contains the 
# normalized number of time each pair of alphabet appears together
# Ex. {'АБ':234,'КЛ':2343,'ЖЗ':23 ..}
def normalized_score_params_on_cipher(text):
    scoring_params = {}
    data = list(text.strip())
    for i in range(len(data)-1):
        alpha_i =data[i].upper()
        alpha_j = data[i+1].upper()
        if alpha_i not in alphabet_list and alpha_i != " ":
            alpha_i = " "
        if alpha_j not in alphabet_list and alpha_j != " ":
            alpha_j = " "
        key = alpha_i+alpha_j
        if key in scoring_params:
            scoring_params[key]+=1
        else:
            scoring_params[key]=1
    # normalization
    appearance_num = scoring_params.values()
    total_appearance_num = sum(appearance_num)
    scoring_params = {k: v / total_appearance_num for k, v in scoring_params.items()}
    return scoring_params

# This function takes the text to be decrypted and a cipher to score the cipher.
# This function returns the log(score) metric
def get_cipher_score(text,cipher,scoring_params):
    cipher_dict = create_cipher_dict(cipher)
    decrypted_text = apply_cipher_on_text(text,cipher)
    scored_f = score_params_on_cipher(decrypted_text)
    cipher_score = 0
    for k,v in scored_f.items():
        if k in scoring_params:
            cipher_score += v*math.log(scoring_params[k])
    return cipher_score

def normalized_get_cipher_score(text,cipher,scoring_params):
    cipher_dict = create_cipher_dict(cipher)
    decrypted_text = apply_cipher_on_text(text,cipher)
    # scored_f = score_params_on_cipher(decrypted_text)
    scored_f = normalized_score_params_on_cipher(decrypted_text)
    cipher_score = 0
    for k,v in scored_f.items():
        if k in scoring_params:
            # cipher_score += v*math.log(scoring_params[k])
            cipher_score += v*scoring_params[k]
    return cipher_score

In [16]:
# Generate a proposal cipher by swapping letters at two random location
def generate_cipher(cipher):
    pos1 = random.randint(0, len(list(cipher))-1)
    pos2 = random.randint(0, len(list(cipher))-1)
    if pos1 == pos2:
        return generate_cipher(cipher)
    else:
        cipher = list(cipher)
        pos1_alpha = cipher[pos1]
        pos2_alpha = cipher[pos2]
        cipher[pos1] = pos2_alpha
        cipher[pos2] = pos1_alpha
        return "".join(cipher)

# Toss a random coin with probability of head p. If coin comes head return true else false.
def random_coin(p):
    unif = random.uniform(0,1)
    if unif>=p:
        return False
    else:
        return True

In [24]:
# Takes as input a text to decrypt and runs a MCMC algorithm for n_iter. Returns the state having maximum score and also
# the last few states 
def MCMC_decrypt(n_iter,cipher_text,scoring_params):
    current_cipher = alphabet_list
    state_keeper = set()
    best_state = ''
    score = 0
    for i in range(n_iter):
        state_keeper.add(current_cipher)
        proposed_cipher = generate_cipher(current_cipher)
        score_current_cipher = get_cipher_score(cipher_text,current_cipher,scoring_params)
        score_proposed_cipher = get_cipher_score(cipher_text,proposed_cipher,scoring_params)
        acceptance_probability = min(1,math.exp(score_proposed_cipher-score_current_cipher))
        if score_current_cipher>score:
            best_state = current_cipher
        if random_coin(acceptance_probability):
            current_cipher = proposed_cipher
        if i%2000==0:
            print("iter",i,":",apply_cipher_on_text(cipher_text,current_cipher)[0:99])
    return state_keeper,best_state

def MCMC_decrypt_norm(n_iter,cipher_text,scoring_params):
    current_cipher = alphabet_list
    state_keeper = set()
    best_state = ''
    score = 0
    for i in range(n_iter):
        state_keeper.add(current_cipher)
        proposed_cipher = generate_cipher(current_cipher)
        score_current_cipher = normalized_get_cipher_score(cipher_text,current_cipher,scoring_params)
        score_proposed_cipher = normalized_get_cipher_score(cipher_text,proposed_cipher,scoring_params)
        acceptance_probability = min(1,math.exp(score_proposed_cipher-score_current_cipher))
        if score_current_cipher>score:
            best_state = current_cipher
        if random_coin(acceptance_probability):
            current_cipher = proposed_cipher
        if i%2000==0:
            print("iter",i,":",apply_cipher_on_text(cipher_text,current_cipher)[0:99])
    return state_keeper,best_state

In [ ]:
## Run the Main Program:
path_to_language_corpus_fold = '/content/drive/My Drive/Magistracy_SPbU/3_semester/Machine_Learning/MCMC/MCMC_decryption/'
scoring_params = create_scoring_params_dict(path_to_language_corpus_fold + 'war_and_peace_ru.txt')

cipher_text = "щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд \
бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз"

print("Text To Decode:", cipher_text)
print("\n")
states,best_state = MCMC_decrypt(2300000,cipher_text,scoring_params)
print("\n")
print("Decoded Text:",apply_cipher_on_text(cipher_text,best_state))
print("\n")
print("MCMC KEY FOUND:",best_state)

Text To Decode: щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз


iter 0 : ЩДШЭСЗДБГДТИЫЛЪИРДВСДПИРВЯСНДЖДЯГФГДОГЪЗД БГДСЫНЪЬИСЗДЕГЯСЗДРГСДОВЯГ БИЪЬИСЗДНТДБНЁДЩДЫИФВСИАДШСВДС
iter 500 : ГОБПИЛОДНОМСЯЖАСУОВИОЭСУВШИРОЕОШНЫНОКНАЛО ДНОИЯРАЙСИЛОЦНШИЛОУНИОКВШН ДСАЙСИЛОРМОДРФОГОЯСЫВИСЬОБИВОИ
iter 1000 : ПОБЫИЛОЕНОМСЯЧАСУОВИОГСУВШИРОТОШНЬНОДНАЛО ЕНОИЯРАЙСИЛОЩНШИЛОУНИОДВШН ЕСАЙСИЛОРМОЕРХОПОЯСЬВИСКОБИВОИ
iter 1500 : ПОБЫЕЛОИНОМСЯЩАСУОВЕОЙСУВШЕРОТОШНЬНОДНАЛО ИНОЕЯРАХСЕЛОЧНШЕЛОУНЕОДВШН ИСАХСЕЛОРМОИРЦОПОЯСЬВЕСКОБЕВОЕ
iter 2000 : ГОБЫЕЛОИНОМСЯЩАСУОВЕОЙСУВШЕДОТОШНЬНОРНАЛО ИНОЕЯДАХСЕЛОЧНШЕЛОУНЕОРВШН ИСАХСЕЛОДМОИДЦОГОЯСЬВЕСКОБЕВОЕ
iter 2500 : ГОЗЫЕЛОИНОМСЯЖАСУОВЕОЙСУВШЕДОТОШНЬНОРНАЛО ИНОЕЯДАХСЕЛОЧНШЕЛОУНЕОРВШН ИСАХСЕЛОДМОИДЦОГОЯСЬВЕСКОЗЕВОЕ
iter 3000 : ГОЗЫЕЛОИНОМСЯЖАСУОВЕОЙСУВ

In [25]:
## Run the Main Program:
path_to_language_corpus_fold = '/content/drive/My Drive/Magistracy_SPbU/3_semester/Machine_Learning/MCMC/MCMC_decryption/'
scoring_params = create_normalized_scoring_params_dict(path_to_language_corpus_fold + 'war_and_peace_ru.txt')

cipher_text = "щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд \
бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз"

print("Text To Decode:", cipher_text)
print("\n")
states,best_state = MCMC_decrypt_norm(10000,cipher_text,scoring_params)
print("\n")
print("Decoded Text:",apply_cipher_on_text(cipher_text,best_state))
print("\n")
print("MCMC KEY FOUND:",best_state)

Text To Decode: щдшэсздбгдтиылъирдвсдпирвясндждягфгдогъзд бгдсынъьисздегясздргсдовяг биъьисзднтдбнёдщдыифвсиадшсвдсвдтиыифислоиадмвжэмиадориъгаджижд бгдмыгъясиорщрвяздбгжвсвывцдявфясогббвясзадндодыгтэрзсисгддвънбдшг въибдмыншг дъвоврзбвдяжыв бвхвдыит гыидолёвънсдщдбнюнцджиждпгдусвдмврэшнрвяз


iter 0 : ЩДШЭСЗДБГДТИЫЛЪИРДВСДПИРВЯСНДЖДЯГКГДОГЪЗД БГДСЫНЪЬИСЗДЕГЯСЗДРГСДОВЯГ БИЪЬИСЗДНТДБНЁДЩДЫИКВСИАДШСВДС
iter 2000 : ЕФЯЁВШФНОФГДХРЛДЧФЩВФСДЧЩУВАФИФУОЪОФЗОЛШФ НОФВХАЛПДВШФЬОУВШФЧОВФЗЩУО НДЛПДВШФАГФНАТФЕФХДЪЩВДМФЯВЩФВ
iter 4000 : ЫЛСЁНЯЛЕПЛАЭУРШЭДЛЧНЛОЭДЧМНТЛЦЛМПИПЛХПШЯЛ ЕПЛНУТШКЭНЯЛЬПМНЯЛДПНЛХЧМП ЕЭШКЭНЯЛТАЛЕТЙЛЫЛУЭИЧНЭЮЛСНЧЛН
iter 6000 : ТВХЭУЦВЁЗВЩМЫЧЮМКВЬУВГМКЬЙУЪВРВЙЗЯЗВНЗЮЦВ ЁЗВУЫЪЮЕМУЦВОЗЙУЦВКЗУВНЬЙЗ ЁМЮЕМУЦВЪЩВЁЪСВТВЫМЯЬУМШВХУЬВУ
iter 8000 : ПГЫАНТГВОГДЯЧБЪЯЩГЦНГЕЯЩЦЙНШГЗГЙОЮОГУОЪТГ ВОГНЧШЪЭЯНТГФОЙНТГЩОНГУЦЙО ВЯЪЭЯНТГШДГВШЖГПГЧЯЮЦНЯКГЫНЦГН


Decoded Text: ЧЫИЛБДЫЯКЫСФЭЩЁФТЫЖБЫЬФТЖОБРЫЪЫОКАКЫПКЁДЫ ЯКЫБЭРЁЙФБДЫЮКОБДЫТКБЫПЖОК ЯФЁЙФБДЫРСЫЯРЦЫЧЫЭФАЖБФВЫИБЖЫБЖЫСФЭФАФБЩПФВЫХЖЪЛХФВЫПТФЁКВЫЪФЪЫ